### Plot survival curves for responds/not responds

In [ ]:
# I have a list of participants
import pandas as pd
data = pd.read_csv('ovarian_parpis_response_wcutoff.csv')

In [ ]:
data

In [ ]:
data[['participant_id','tumour_sample_platekey','germline_sample_platekey']]

In [ ]:
%run /home/sagemaker-user/genomics-gel/experimental/2024_11_27_end2end_gel_cohorts/s3_helper.py
rdl_s3_path = "s3://676167504555-gel-structured-data-prod/RESEARCH_ENVIRONMENT_1/RR18/100K_GEL_Cancer_Analysis/2022/10/13/100K_GEL_Cancer_Analysis/data.tsv"
ca = read_tsv_from_rdl(rdl_s3_path)

In [ ]:
# rdl = get_rdl_catalogue()

In [ ]:
# rdl[(rdl['Key'].str.contains('ONS')) & (rdl['Key'].str.contains('RR19'))]['Key'].tolist()

In [ ]:
# hes_tables=[
# 			('RESEARCH_ENVIRONMENT_1/RR19/100K_NHSD_HES_APC/2024/03/03/100K_NHSD_HES_APC/data.tsv','epistart'),
# 			('RESEARCH_ENVIRONMENT_1/RR19/100K_NHSD_HES_AE/2023/12/11/100K_NHSD_HES_AE/data.tsv','arrivaldate'), 
# 			('RESEARCH_ENVIRONMENT_1/RR19/100K_NHSD_HES_OP/2024/03/03/100K_NHSD_HES_OP/data.tsv','apptdate'), 
# 			('RESEARCH_ENVIRONMENT_1/RR19/100K_NHSD_HES_CC/2024/03/03/100K_NHSD_HES_CC/data.tsv','ccdisdate')
# 			]

In [ ]:
# h = read_tsv_from_rdl(hes_tables[0][0])

In [ ]:
# h = read_tsv_from_rdl(hes_tables[0][0])[hes_tables[0][1]]

In [ ]:
# d = 'RESEARCH_ENVIRONMENT_1/RR19/100K_GEL_Cancer_Tumour/2024/06/12/100K_GEL_Cancer_Tumour/data.tsv'
# data = read_tsv_from_rdl(d)
# data[['participant_id', 'diagnosis_date']]

In [ ]:
data.columns

### What we need:
- date of diagnosis
- date of death
- survival
- plot

In [ ]:
import sys
from pathlib import Path

# Get the absolute path to the other repo
parent_folder = Path.cwd().parent.parent.parent
repo2_path = parent_folder / "survival_python" / "src"

# # Add the other repo to Python's path
sys.path.append(str(repo2_path))

In [ ]:
%run /home/sagemaker-user/survival_python/src/gelpack/survival.py

In [ ]:
ca = ca.merge(data[['participant_id','tumour_sample_platekey']], how='inner', on =['participant_id','tumour_sample_platekey'])

In [ ]:
ca

In [ ]:
c = Survdat(
		ca,
		ca['participant_id'], 
		'1', 
		'False')

In [ ]:
Survdat

In [ ]:
c.quer_ons()

In [ ]:
c.quer_hes(sample=False)

In [ ]:
c.quer_dod()

In [ ]:
c.get_earliest_cancer_diagnosis()

In [ ]:
c.get_sample_diagnosis_date()

In [ ]:
c.surv_time()

In [ ]:
c.surv_dat

In [ ]:
c.surv_dat.participant_id.nunique()

In [ ]:
surv_dat = c.surv_dat.merge(data, on=['participant_id'], how='left', suffixes=('', '_manual'))

In [ ]:
surv_dat

In [ ]:
surv_dat.respond.value_counts()

In [ ]:
grouped_dat = assign_groups(
                dataframe=surv_dat, 
                vars=['respond'], 
                type='and'
                )
# map_dict = {'group_1':'Mut','group_2':'WT'}
map_dict = {'group_1':'Responds','group_2':'No Response'}

In [ ]:
dat = km_survival(
		data=grouped_dat,
		strata=map_dict.values(),
		map_dict=map_dict,
		output='output',
		plt_title='Survival Curves for patients treated with PARPi',
		plotting='show'
		)